# Fetching player stats from current season


In [1]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import pandas as pd
import time

In [3]:
def getPlayerStats(playerID):
    playerStats = playercareerstats.PlayerCareerStats(per_mode36='PerGame',player_id=playerID)
    playerStats_df = playerStats.get_data_frames()[0]
    return playerStats_df

In [34]:
lebron_stats = getPlayerStats(2544) #lebron james stats
current_season = lebron_stats[len(lebron_stats)-1:]
current_season


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
21,2544,2024-25,00,1610612747,LAL,40.0,70,70,34.9,9.3,...,0.782,1.0,6.8,7.8,8.2,1.0,0.6,3.7,1.4,24.4


In [5]:
sga_stats = getPlayerStats(1628983) #free throw merchant stats
season_rankings = playercareerstats.PlayerCareerStats
current_season = sga_stats[len(sga_stats)-1:]
current_season.to_csv(f"sga_stats.csv",index=False)

In [8]:
print(get_player_stats_for_season(2544,2024))

    PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
21       2544   2024-25        00  1610612747               LAL        40.0   

    GP  GS   MIN  FGM  ...  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  \
21  70  70  2444  651  ...   0.782    72   474  546  575   70   39  260  99   

     PTS  
21  1710  

[1 rows x 27 columns]


In [ ]:
def get_player_stats_for_season(player_id, season_year):
    try:
        stats = playercareerstats.PlayerCareerStats(player_id=player_id)
        df = stats.get_data_frames()[0]
        df = df[df['SEASON_ID']==f"{season_year}-{str(season_year+1)[-2::]}"]
        df = df[df['LEAGUE_ID']=='00']
        df['PPG'] = round(df['PTS'] / df['GP'],1)
        df['APG'] = round(df['AST'] / df['GP'],1)
        df['PER'] = round((df['FGM']*85.91 + df['STL'] * 53.897 + df['FG3M']*51.757 + df['FTM']*46.845 + df['BLK']*39.19+df['OREB']*39.19+df['AST']*34.677+df['DREB']*14.707-df['PF']*17.174-(df['FTA']-df['FTM'])*20.091-(df['FGA']-df['FGM'])*39.190-df['TOV']*53.897) * (1/df['MIN']),1)
        return df
    except Exception as e:
        print(f"Failed to get stats for player {player_id}: {e}")
        return pd.DataFrame()


def get_all_player_stats(season_year):
    all_players = players.get_active_players()
    all_stats = []
    for i, player in enumerate(all_players):
        print(f"[{i+1}/{len(all_players)}] Fetching stats for {player['full_name']} {player['id']}")
        stats_df = get_player_stats_for_season(player['id'], season_year)
        if not stats_df.empty:
            if len(stats_df)>1:
                stats_df.at[stats_df.index[-1], 'TEAM_ID'] = stats_df.at[stats_df.index[-2], 'TEAM_ID']
                stats_df.at[stats_df.index[-1], 'TEAM_ABBREVIATION'] = stats_df.at[stats_df.index[-2], 'TEAM_ABBREVIATION']
            stats_df['FULL_NAME'] = player['full_name']
            all_stats.append(stats_df.last_valid_index())
        time.sleep(0.6)
    final_df = pd.concat(all_stats, ignore_index=True)
    return final_df

season_year = 2024
regular_df = get_all_player_stats(season_year)
regular_df = regular_df.drop_duplicates(subset=['PLAYER_ID', 'SEASON_ID'], keep='last')

regular_df.to_csv(f"player_stats_{season_year}.csv", index=False)

[1/572] Fetching stats for Precious Achiuwa 1630173
[2/572] Fetching stats for Steven Adams 203500
[3/572] Fetching stats for Bam Adebayo 1628389
[4/572] Fetching stats for Ochai Agbaji 1630534
[5/572] Fetching stats for Santi Aldama 1630583
[6/572] Fetching stats for Trey Alexander 1641725
[7/572] Fetching stats for Nickeil Alexander-Walker 1629638
[8/572] Fetching stats for Grayson Allen 1628960
[9/572] Fetching stats for Jarrett Allen 1628386
[10/572] Fetching stats for Jose Alvarado 1630631
[11/572] Fetching stats for Kyle Anderson 203937


KeyError: 1